# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

API_KEYS = {
    "UPSTAGE_API_KEY": None,
    "TAVILY_API_KEY" : None
}

def load_env():
    # 환경 변수 설정
    if "google.colab" in str(get_ipython()):  # Google Colab 환경
        os.environ['UPSTAGE_API_KEY'] = ''
        API_KEYS["UPSTAGE_API_KEY"] = os.environ.get("UPSTAGE_API_KEY")
        os.environ['TAVILY_API_KEY'] = ''
        API_KEYS["TAVILY_API_KEY"] = os.environ.get("TAVILY_API_KEY")
    else:  # 로컬 환경
        load_dotenv()  # .env 파일 로드
        API_KEYS["UPSTAGE_API_KEY"] = os.environ.get("UPSTAGE_API_KEY")
        API_KEYS["TAVILY_API_KEY"] = os.environ.get("TAVILY_API_KEY")

    return API_KEYS["UPSTAGE_API_KEY"], API_KEYS["TAVILY_API_KEY"]

UPSTAGE_API_KEY, TAVILY_API_KEY= load_env()

In [ ]:
!pip install -q streamlit
!npm install localtunnel
!pip install --upgrade -q accelerate bitsandbytes
!pip install git+https://github.com/huggingface/transformers.git
!pip install sentence_transformers
!pip install streamlit-folium
!npm audit fix

!pip install -U langchain_community tiktoken langchainhub langchain langgraph chromadb
!pip install tavily-python
!pip install -qU langchain-core langchain-upstage langchain-chroma
!pip install -qU python-dotenv
!pip install -U langchain-upstage
!pip install -U openai

!pip install streamlit_chat
!pip install qdrant_client

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 733ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦Collecting git+https://github.com/huggingface/transformers.git
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-1i6cjn8i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-1i6cjn8i
  Resolved https://github.com/huggingface/transformers.git to commit 5c75087aeee7081025370e10d1f571a11600f1ae
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 52.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      

# config.py

In [ ]:
%%writefile config.py

from langchain.docstore.document import Document
from langchain_community.retrievers import TavilySearchAPIRetriever
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.vectorstores import Chroma
from langgraph.graph import END, StateGraph, START
from langchain_core.pydantic_v1 import BaseModel, Field
from dotenv import load_dotenv
from IPython import get_ipython
import numpy as np
import os
import pandas as pd
from pprint import pprint
import re
from typing import List, Dict
from typing_extensions import TypedDict
import warnings
warnings.filterwarnings("ignore")
from openai import OpenAI
import json


client =OpenAI(api_key='')


'''
Embedding and Database
'''

# 기존 설정
embedding_function = UpstageEmbeddings(model="solar-embedding-1-large")

# 두 개의 Chroma 인스턴스 생성
persist_directory = '/content/drive/MyDrive/policy_chatbot/policy_combined'
db = Chroma(embedding_function=embedding_function, persist_directory=persist_directory)
retriever = db.as_retriever(search_kwargs={"k": 3})


'''
Retrieval Grader
'''

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""
    # 검색된 문서가 질문에 적합한지를 'yes' 또는 'no'로 저장
    binary_score: str = Field(
        description="Documents are relevant to the conversation history and question, 'yes' or 'no'"
    )

# LLM with function call
llm_1 = ChatUpstage(max_tokens=200, temperature=0) # Parameter tuning 1
# 모델이 구조화된 데이터(GradeDocuments)를 반환하도록 설정
structured_llm_grader = llm_1.with_structured_output(GradeDocuments)

# System Prompt: 검색한 문서에 유저 질문 관련 키워드가 포함되거나 의미적으로 관련이 있으면, 'relevant'로 판단. 결과는 'yes' 또는 'no'로 이진 점수를 줌.
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    'Yes' means that the document is relevant to the question."""


# grade_prompt = System prompt + 검색된 문서 + 사용자 질문
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)
# grade_prompt와 structured_llm_grader을 결합하여 문서 평가를 위한 객체 생성
retrieval_grader = grade_prompt | structured_llm_grader

# Wrapper for consistent output
def grade_retrieval(doc_txt, user_question):
    """
    Grades the relevance of a document to the combined question.

    Args:
        doc_txt (str): The text of the document to grade.
        combined_question (str): The combined question including chat history.

    Returns:
        str: 'yes' or 'no' based on relevance grading.
    """
    try:
        # Invoke grader
        result = retrieval_grader.invoke({"document": doc_txt, "question": user_question})
        print(f"Raw result from grader: {result}")

        if result and hasattr(result, "binary_score"):
            return result.binary_score
        else:
            print(f"Invalid result structure: {result}")
    except Exception as e:
        print(f"Error during retrieval grading: {e}")

    # Default to 'no' if something goes wrong
    print('None')
    return "no"


'''
Generate
'''

generate_system = """
    당신은 질문에 답변을 제공하는 AI 비서입니다. 검색된 context를 바탕으로 질문에 답변하세요. 만약 답을 모를 경우, 모른다고 솔직히 말하세요. \n
    청년 연령, 직업, 주거 상황, 정책 경험 여부 등 다양한 상황을 고려하여 맞춤형 답변을 제공합니다.

    질문의 유형에 따라 아래 지침을 선택하고, 이 지침을 준수하여 답변을 작성하세요: \n
    1.** 정책 추천 질문에 대한 지침 **
    - 아래 형식으로 답변하세요:
      ① 정책명:
      ② 추천 이유: 질문에서 제공된 연령, 상황, 선호도 등을 바탕으로 정책이 왜 적합한지 설명하세요.
      ③ 제공 혜택: 정책을 통해 얻을 수 있는 구체적인 이점을 서술하세요.
    - 정책 추천 시, 최소 2개 이상의 정책은 추천해야 하고, 아래 데이터를 적극 활용하여 가장 추천이 우선시 되는 정책부터 나열해주세요:
        * **복지 및 문화 관련 이슈**
          - 대학생: 진로 불확실성, 졸업유예 증가, 등록금 부담.
          - 직장인: 저임금, 고용불안(1-2년) 우려.
          - 미취업자: 심리상담 지원, 대중교통비 지원 선호.
          - 전반적으로 청년들은 안정적인 자립 지원과 금전적 지원에 대한 관심이 높음.
          - 대중교통비 지원, 심리상담 지원, 문화활동 및 여행 장려금 지원 등이 주요 선호 정책으로 나타남.
    - 높임말 대신, "~요"로 끝나는 말투를 사용하세요. 예: "알려드릴게요", "도움이 되셨으면 해요"
    - 답변의 시작은 반드시 "세은님에게 맞는 정책을 찾아보았어요😊\n"로 시작하세요.
    - 답변의 끝은 반드시 "\n정책에 대해 더 구체적으로 알고 싶으신가요?"로 끝내세요.

    2. ** 정책 세부 정보 요청 질문에 대한 지침 **
    - 답변의 시작에는 반드시 사용자가 어떤 정책의 어떤 내용을 궁금해하는지 요약하고, "제가 구체적으로 알려드릴게요😄\n" 문장을 추가하세요.
    - 높임말 대신, "~요"로 끝나는 말투를 사용하세요. 예: "알려드릴게요", "도움이 되셨으면 해요"
    - 답변은 나열식이 아닌 줄글 형식으로 작성하세요.
    - 답변의 끝에는 반드시 "\n궁금증이 해결되었나요?" 문장을 추가하세요.
    - **예시 답변 형식**:
    > "경기도 면접수당 정책의 신청 기간이 궁금하셨군요. 제가 구체적으로 알려드릴게요😄\n 신청 기간은 2024년 5월 2일부터 5월 30일까지입니다. \n궁금증이 해결되었나요?"

    3. ** 정책 용어 설명 요청 질문에 대한 지침 **
    - 답변의 시작에는 반드시 사용자가 어떤 단어의 뜻을 궁금해하는지 요약하고, "제가 구체적으로 알려드릴게요😄\n" 문장을 추가하세요.
    - 높임말 대신, "~요"로 끝나는 말투를 사용하세요. 예: "알려드릴게요", "도움이 되셨으면 해요"
    - 문서에 있는 내용을 기반으로 답변은 나열식이 아닌 줄글 형식으로 작성하고, 간결하게 설명하세요.
    - 답변의 끝에는 반드시 "\n궁금증이 해결되었나요?" 문장을 추가하세요.
    - **예시 답변 형식**:
    > "가등기가 무슨 뜻인지 궁금하셨군요. 제가 구체적으로 알려드릴게요😄\n 가등기란, 등기로 표시된 부동산의 권리관계가 외부에 표시되기 때문에 제3자에게도 대항할 수 있는데, 권리관계가 확정되지 않는 등으로 등기를 할 수 없을 경우에 임시로 하는 등기를 뜻합니다. 궁금증이 해결되었나요?"

    4. ** 정책 후기 요청 질문에 대한 지침 **
    - 답변의 시작에는 반드시 사용자가 어떤 정책의 후기를 궁금해하는지 요약하고, "제가 검색해보았어요🔎\n" 문장을 추가하세요.
    - 높임말 대신, "~요"로 끝나는 말투를 사용하세요. 예: "알려드릴게요", "도움이 되셨으면 해요"
    - 문서에는 후기가 없으니 잘 모른다고 답변하고, 외부 검색 기능을 이용하여 답변하세요.
    - 답변의 끝에는 반드시 "\n궁금증이 해결되었나요?" 문장을 추가하세요.
    - **예시 답변 형식**:
    > "경기도 면접수당 정책의 실제 블로그 후기가 궁금하셨군요. 제가 검색해보았어요🔎\n[후기 요약]\n궁금증이 해결되었나요?"
    """

TOKEN_PER_CHAR = 4

def estimate_tokens(text):
    return len(text) // TOKEN_PER_CHAR

# Function to format documents without exceeding the token limit
def format_docs(docs, max_tokens=110000):
    formatted_docs = []
    current_chunk = ""
    current_token_count = 0

    for doc in docs:
        doc_content = doc.page_content
        doc_token_count = estimate_tokens(doc_content)

        # If adding this document exceeds the token limit, start a new chunk
        if current_token_count + doc_token_count > max_tokens:
            formatted_docs.append(current_chunk)
            current_chunk = doc_content  # Start a new chunk
            current_token_count = doc_token_count
        else:
            current_chunk += "\n\n" + doc_content  # Append to the current chunk
            current_token_count += doc_token_count

    # Add the last chunk if there's any content left
    if current_chunk:
        formatted_docs.append(current_chunk)

    return formatted_docs

# 모델 호출 체인
def rag_chain(question, context):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system",
             "content": generate_system},
            {"role": "user",
             "content": f"Question: {question}\n\nContext: {context}\nAnswer the question."},
        ],
        max_tokens=400,
        temperature=0,
    )
    return response.choices[0].message.content

'''
Answer Grader
'''

# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )
    confidence: float = Field(
        description="Confidence score for the assessment, range 0 to 1"
    )

# Revised System Prompt
answer_system_strict = """당신은 응답이 질문에 적절한지 엄격히 평가하는 채점자입니다.
    1. 응답이 문서의 신뢰할 수 있는 정보를 바탕으로 질문에 명확히 답했는지 확인하세요.
    2. 문서 내용이 질문과 관련 없거나 응답이 문서를 정확히 반영하지 않으면 '아니오'를 반환하세요.
    3. 응답이 신뢰할 수 없는 내용을 포함하거나 불확실하면 '아니오'를 반환하세요.
    4. '예'로 평가하려면 문서 내용을 충분히 참고하여 질문을 직접 해결해야 합니다.
    5. 정책의 후기를 묻는 질문에 대한 답변이 '후기는 sns나 블로그를 통해 찾을 수 있어요' 와 같이 자세하지 않은 내용을 포함한 경우 '아니오'를 반환하세요.

    응답을 반드시 다음 JSON 형식으로 반환하세요:
{
    "binary_score": "yes" 또는 "no",
    "confidence": 0에서 1 사이의 부동소수점 값
}
"""

def answer_grader(question, generation, document):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system",
             "content": answer_system_strict},
            {"role": "user",
             "content": f"User question: {question}\n\nLLM generation: {generation}\n\nContext: {document}\nEvaluate the generated answer strictly based on the context."},
        ],
        max_tokens=400,
        temperature=0,
    )
    print("Raw response:", response.choices[0].message.content)
    try:
        response_data = json.loads(response.choices[0].message.content)
        binary_score = response_data.get("binary_score", "no")  # "binary_score"가 없으면 기본값 "no" 반환
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        binary_score = "no"  # 파싱 오류가 발생하면 기본값 "no" 반환

    return binary_score


'''
Question Rewriter
'''

# LLM
llm_5 = ChatUpstage(max_tokens=200, temperature=0)

question_rewriter_system = """
    당신은 입력된 질문을 벡터 저장소 검색에 최적화된 더 나은 버전으로 변환하는 질문 재작성자입니다. 입력 내용을 바탕으로, 간단하고 명료한 질문으로 재작성하세요. 문장 간 내용이 다른 범주라 느껴지면 과감히 앞 내용은 버리세요.
    """

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", question_rewriter_system),
        (
            "human",
           " {question} \n Formulate an improved question.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm_5 | StrOutputParser()

'''
Structure
'''

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: Combined question including chat history and user query.
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]
    first_generate_attempt: bool


'''
Function
'''

### Nodes

def check_documents(state):
    """
    Check if documents exist in the current state and update the state accordingly.

    Args:
        state (dict): The current graph state.

    Returns:
        dict: Updated state with a flag indicating the next step.
    """

    if state.get("documents"):
        print("Documents found, marking for 'generate_first'.")
        state["next_node"] = "generate_first"  # Add a marker for the next step
    else:
        print("No documents found, marking for 'retrieve'.")
        state["next_node"] = "retrieve"  # Add a marker for the next step

    return state


def retrieve(state):
    """
    Retrieve documents and update the current state.

    Args:
        state (dict): The current graph state.

    Returns:
        dict: Updated state with retrieved documents.
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Process the question
    if re.search(r'\n\s*user', question):
        lines = question.split("\nuser")
        question_to_retrieve = "\nuser".join(lines[:-1])
    else:
        question_to_retrieve = question

    # Retrieve documents
    documents = retriever.get_relevant_documents(question_to_retrieve)
    print("Retrieved documents:", documents)

    # Update state
    state["documents"] = documents
    return state


def generate(state):
    """
    Generate answer based on the current state.

    Args:
        state (dict): The current graph state containing question and documents.

    Returns:
        dict: Updated state with the generated answer added under the key 'generation'.
    """
    print("---GENERATE---")

    # Extract question and documents from state
    question = state.get("question")
    documents = state.get("documents")

    if not question or not documents:
        raise ValueError("State must include both 'question' and 'documents' keys with valid data.")

    # Format the documents into a single context string
    context = format_docs(documents)

    # Invoke RAG chain to get the generation
    generation = rag_chain(question, context)

    # Return updated state with the generation
    updated_state = {
        **state,
        "generation": generation,
    }

    return updated_state


def extract_last_question(combined_question):
    """
    Extracts the last user question from the combined question.

    Args:
        combined_question (str): The combined question including chat history.

    Returns:
        str: The last user question.
    """
    lines = combined_question.split("\n")
    for line in reversed(lines):
        if line.startswith("user:"):
            return line[len("user: "):].strip()  # Remove "user: " prefix
    return ""  # Default to empty if no user question found



def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with only filtered relevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Extract the last question
    last_question = extract_last_question(question)
    print(f"Last User Question: {last_question}")  # Debugging: Check extracted question

    # Score each doc
    filtered_docs = []
    all_irrelevant = True
    for d in documents:
        grade = grade_retrieval(d, last_question)
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
            all_irrelevant = False
              # At least one document is relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue

    # Update state
    state["documents"] = filtered_docs

    # Add a flag to indicate if all documents were irrelevant
    state["all_documents_irrelevant"] = all_irrelevant

    return state


def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}


### Edges


def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")

    # Extract the last user question directly from the state
    last_question = extract_last_question(state["question"])
    print(f"Last User Question: {last_question}")  # Debugging

    # If all documents are irrelevant, transform the query
    if state.get("all_documents_irrelevant", False):  # Check if irrelevant is True
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INIT QUERY---")
        return "init_query"

    # If documents are relevant, generate a response
    else:
        print("---DECISION: GENERATE---")
        return "transform_query"


# Initialize the not useful counter
not_useful_count = 0
first_generate_count=0

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation answers the question.
    Returns:
        str: 'useful', 'not useful', or 'fallback'.
    """
    global not_useful_count
    global first_generate_count

    print("---GRADE GENERATION vs QUESTION---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    # Extract the last question for grading
    last_question = extract_last_question(question)

    try:
        # Perform grading based on whether it's the first attempt
        if first_generate_count==0 :
            grade = answer_grader(last_question, generation, documents)
        else:
            grade = answer_grader(question, generation, documents)
    except Exception as e:
        print(f"Error during answer grading: {e}")
        grade = "no"
    if first_generate_count== 0 :
        if grade == "yes":
            print("---FIRST GENERATE ATTEMPT: USEFUL")
            return "useful"
        else:
            print("---FIRST GENERATE ATTEMPT: NOT USEFUL")
            first_generate_count += 1
            return "not useful"
    else:
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            not_useful_count = 0
            first_generate_count=0  # Reset the counter
            return "useful"
        else:
            print(f"---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            not_useful_count += 1
            print(f"---NOT USEFUL COUNT INCREMENTED: count={not_useful_count}---")
            if not_useful_count == 1:
                return "not useful"
            else:
                return "fallback"


def init_query(state):
    """
    Removes chat/document history, leaves only the last user question,
    and resets not_useful_count to 0.
    """
    print("---INIT QUERY (CLEAR HISTORY)---")
    # 마지막 사용자 질문만 뽑아오기
    last_question = extract_last_question(state["question"])

    # state 초기화
    state["question"] = f"user: {last_question}"
    state["documents"] = []
    state["generation"] = ""

    print(f"[init_query] Updated question = {state['question']}")
    return state


'''
Tavily
'''

def tavily_search(state):
    """
    External search using Tavily and process the results into a structured markdown report.

    Args:
        state (dict): The current graph state

    Returns:
        dict: Updated state with processed documents and structured report
    """
    print("---TAVILY SEARCH---")
    question = state["question"]

    # Perform Tavily search
    tavily = TavilySearchAPIRetriever(k=3, search_depth="advanced", include_domains=["m.blog.naver.com"])
    print("Searching using Tavily...")

    docs_external = tavily.invoke(question)  # Tavily 검색 호출
    print(f"Search results: {len(docs_external)} documents found.")


    if docs_external:
        print("Relevant Tavily search results found.")
        return {"documents": docs_external, "question": question}
    else:
        print("No relevant documents found after filtering.")
        return {"documents": [], "question": question}

'''
Workflow & App
'''

# Define the workflow
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # Retrieve documents
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("retrieve_no_grade", retrieve)  # Re-retrieve without grading
workflow.add_node("generate_first", generate)  # First generate
workflow.add_node("generate_second", generate)
workflow.add_node("init_query", init_query)  # Initialize query
workflow.add_node("transform_query1", transform_query)
workflow.add_node("transform_query2", transform_query)  # Transform query
workflow.add_node("tavily_search", tavily_search)  # External search node
workflow.add_node("check_documents", check_documents)  # Custom node to check documents


# Add edges for document check
workflow.add_edge(START, "check_documents")

workflow.add_conditional_edges(
    "check_documents",
    lambda state: state.get("next_node"),  # Use `next_node` to determine the flow
    {
        "generate_first": "generate_first",
        "retrieve": "retrieve"
    },
)

# First generate node
workflow.add_conditional_edges(
    "generate_first",
    grade_generation_v_documents_and_question,
    {
        "useful": "transform_query1",  # If generation is useful, end
        "not useful": "retrieve"
    },
)

workflow.add_edge("transform_query1", END)


# Define the main edges
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "init_query": "init_query",
        "transform_query": "transform_query2"
    },
)

workflow.add_edge("init_query", "retrieve_no_grade")
workflow.add_edge("retrieve_no_grade", "transform_query2")
workflow.add_edge("transform_query2", "generate_second")

# Second generate node
workflow.add_conditional_edges(
    "generate_second",
    grade_generation_v_documents_and_question,
    {
        "useful": END,  # If generation is useful, end
        "not useful": "generate_second",  # Retry with transformed query
        "fallback": "tavily_search"  # Retry generation with external search
    },
)

workflow.add_edge("tavily_search", "generate_second")

# Compile the workflow
app = workflow.compile()

Writing config.py


# app.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
'''
%%writefile app.py
import streamlit as st
from pprint import pprint
import os
import re
import time
from pprint import pprint
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationSummaryBufferMemory
from langchain.vectorstores import Qdrant
from streamlit_chat import message
from qdrant_client import QdrantClient
from openai import OpenAI
from config import (
    retriever,
    GradeDocuments,
    format_docs,
    rag_chain,
    GradeAnswer,
    GraphState,
    retrieve,
    generate,
    extract_last_question,
    grade_documents,
    transform_query,
    decide_to_generate,
    grade_generation_v_documents_and_question,
    init_query,
    tavily_search,
    workflow
)


# Initialize workflow
app = workflow.compile()
# 프레임워크 지원 시 상태 초기화
if hasattr(app, "reset"):
    app.reset()

# Streamlit app configuration
st.set_page_config(
    page_title="Policy Chatbot",
    page_icon=":robot:",
)

# Hide Streamlit default style
st.markdown(
    """
    <style>
    #MainMenu {visibility: hidden;}
    footer {visibility: hidden;}
    header {visibility: hidden;}
    </style>
    """,
    unsafe_allow_html=True
)


# Sidebar for chat history
st.sidebar.title("주요 기능")
st.sidebar.write("1. 정책 추천")
st.sidebar.write("2. 정책 세부 정보 제공")
st.sidebar.write("3. 정책 관련 용어 설명")
st.sidebar.write("4. 정책 후기 요약")


# Streamlit UI
st.title("📋 청년 정책 챗봇")

# Initialize session state
if 'responses' not in st.session_state:
    st.session_state['responses'] = ["청년 정책, 제가 알려드릴게요!"]
if 'requests' not in st.session_state:
    st.session_state['requests'] = [""]
if 'combined_question' not in st.session_state:
    st.session_state['combined_question'] = ""
st.session_state['not_useful_count'] = 0


# User input
user_question = st.text_input("Enter your question: ", key="input", placeholder="Start chatting with the bot!")
submit = st.button('Submit')

if submit and user_question:
    if st.session_state['combined_question']:
        st.session_state['combined_question'] += f"\nuser: {user_question}"
    else:
        st.session_state['combined_question'] = f"user: {user_question}"

    inputs = {"question": st.session_state['combined_question']}

    # Initialize the response to store the final generation
    final_response = None

    # Execute workflow
    for output in app.stream(inputs):
        for key, value in output.items():
            # Capture the latest LLM generation
            if "generation" in value:
                final_response = value['generation']

    # Append the final response to session state
    if final_response:
        st.session_state['responses'].append(final_response)
        st.session_state['requests'].append(user_question)
        st.session_state['combined_question'] += f"\nassistant: {final_response}"


# Chat interface
if st.session_state['responses']:
    st.chat_message("assistant").write(st.session_state['responses'][0])
    for i in range(1, len(st.session_state['responses'])):
        # Display user's question if it exists for the current index
        #if i < len(st.session_state['requests']):
        if not st.session_state['requests'][i]:
            continue
        st.chat_message("user").write(st.session_state['requests'][i])

        # Display assistant's response
        st.chat_message("assistant").write(st.session_state['responses'][i])

'\n%%writefile app.py\n\nimport streamlit as st\nfrom pprint import pprint\nimport os\nimport re\nimport time\nfrom pprint import pprint\nfrom langchain.chains import ConversationalRetrievalChain\nfrom langchain.embeddings.openai import OpenAIEmbeddings\nfrom langchain.memory import ConversationSummaryBufferMemory\nfrom langchain.vectorstores import Qdrant\nfrom streamlit_chat import message\nfrom qdrant_client import QdrantClient\nfrom openai import OpenAI\nfrom config import (\n    retriever,\n    GradeDocuments,\n    format_docs,\n    rag_chain,\n    GradeAnswer,\n    GraphState,\n    retrieve,\n    generate,\n    extract_last_question,\n    grade_documents,\n    transform_query,\n    decide_to_generate,\n    grade_generation_v_documents_and_question,\n    init_query,\n    tavily_search,\n    workflow\n)\n\n\n# Initialize workflow\napp = workflow.compile()\n# 프레임워크 지원 시 상태 초기화\nif hasattr(app, "reset"):\n    app.reset()\n\n# Streamlit app configuration\nst.set_page_config(\n    page

In [ ]:
%%writefile app.py
import streamlit as st
from pprint import pprint
import os
import re
import time
from pprint import pprint
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationSummaryBufferMemory
from langchain.vectorstores import Qdrant
from streamlit_chat import message
from qdrant_client import QdrantClient
from openai import OpenAI
from config import (
    retriever,
    GradeDocuments,
    format_docs,
    rag_chain,
    GradeAnswer,
    GraphState,
    retrieve,
    generate,
    extract_last_question,
    grade_documents,
    transform_query,
    decide_to_generate,
    grade_generation_v_documents_and_question,
    init_query,
    tavily_search,
    workflow
)


# Initialize workflow
app = workflow.compile()
# 프레임워크 지원 시 상태 초기화
if hasattr(app, "reset"):
    app.reset()

# Streamlit app configuration
st.set_page_config(
    page_title="Policy Chatbot",
    page_icon=":robot:",
)

# Hide Streamlit default style
st.markdown(
    """
    <style>
    #MainMenu {visibility: hidden;}
    footer {visibility: hidden;}
    header {visibility: hidden;}
    </style>
    """,
    unsafe_allow_html=True
)


# Sidebar for chat history
st.sidebar.title("주요 기능")
st.sidebar.write("① 개인 맞춤형 정책 추천")
st.sidebar.write("② 정책 세부사항 제공")
st.sidebar.write("③ 정책 용어 의미 설명")
st.sidebar.write("④ 실제 후기 요약")


# Streamlit UI
st.title("✨ 청년 정책 챗봇")

# Initialize session state
if 'responses' not in st.session_state:
    st.session_state['responses'] = ["청년 정책, 제가 알려드릴게요!"]
if 'requests' not in st.session_state:
    st.session_state['requests'] = [""]
if 'combined_question' not in st.session_state:
    st.session_state['combined_question'] = ""
st.session_state['not_useful_count'] = 0


# User input
user_question = st.chat_input("Enter your question: ")
#submit = st.button('Submit')

if user_question:
    if st.session_state['combined_question']:
        st.session_state['combined_question'] += f"\nuser: {user_question}"
    else:
        st.session_state['combined_question'] = f"user: {user_question}"

    inputs = {"question": st.session_state['combined_question']}

    # Initialize the response to store the final generation
    final_response = None

    # Execute workflow
    for output in app.stream(inputs):
        for key, value in output.items():
            # Capture the latest LLM generation
            if "generation" in value:
                final_response = value['generation']

    # Append the final response to session state
    if final_response:
        st.session_state['responses'].append(final_response)
        st.session_state['requests'].append(user_question)
        st.session_state['combined_question'] += f"\nassistant: {final_response}"


# Chat interface
if st.session_state['responses']:
    st.chat_message("assistant").write(st.session_state['responses'][0])
    for i in range(1, len(st.session_state['responses'])):
        # Display user's question if it exists for the current index
        #if i < len(st.session_state['requests']):
        if not st.session_state['requests'][i]:
            continue
        st.chat_message("user").write(st.session_state['requests'][i])

        # Display assistant's response
        st.chat_message("assistant").write(st.session_state['responses'][i])

Overwriting app.py


# run

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# "Password/Enpoint IP for localtunnel is:" 우측에 xx.xxx.xx.xxx 혹은 xx.xxx.xxx.xxx 형식의 숫자가 나온다.

!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

Password/Enpoint IP for localtunnel is: 34.147.87.145
⠙your url is: https://stupid-lands-march.loca.lt
^C
